<a id='top_cell'></a>

# Advanced Python - Parsing Commandline Arguments
<div style="text-align: right"> [Back to Start](0 Start.ipynb) </div>

Instead of trying to understand and process the commandline options with ``if .. else: ..`` statements, use the standard ``argparse`` Python module with all its benefits. There is also a C-style (``getopt``) parsers for command line options, but we recommend ``argparse``.

Official documentation: [Parser for command-line options, arguments and sub-commands](https://docs.python.org/3/library/argparse.html)

A tutorial from PyMOTW-3: [Commandline Option and argument parsing](https://pymotw.com/3/argparse/)

## Steps to take in setting up commadline arguments

1. Setting up a parser for your commandline
2. Define the mandatory and optional arguments
3. Parse the commandline
4. Use the parsed argument values

In [ ]:
from importlib import reload
import logging

### Setting up a parser

Arguments from the commandline are passed into the variable ``sys.argv``.

In [ ]:
%pwd

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import sys

print(sys.argv)

In [ ]:
%run ../scripts/argparse_example.py

In [ ]:
%run ../scripts/argparse_example.py input.yaml

In [ ]:
%run ../scripts/argparse_example.py -X --config input.yaml --log out.log --logging-level=INFO

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser()
parser.parse_args()

In [ ]:
%run ../scripts/argparse_example.py -X --config input.yaml --log out.log --logging-level=INFO

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")
parser.parse_args()

In [ ]:
%run ../scripts/argparse_example.py -h

### Defining the arguments

When defining arguments, optional arguments start with a hyphen ``'-'`` while positional arguments can be just a name like ``'foo'``.

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', action='store_true')
parser.add_argument('foo')
parser.parse_args()


In [ ]:
%run ../scripts/argparse_example.py -X a_filename.txt

In [ ]:
%run ../scripts/argparse_example.py -h

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', action='store_true', help="use the extended version")
parser.add_argument('foo', help='enter what the foo this is')
parser.parse_args()

In [ ]:
%run ../scripts/argparse_example.py -h

---

**Add a single optional argument**

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', '--eXtended', action='store_true', help="use the extended version")
args = parser.parse_args()

print (f"args = {args}")
print (f"args.eXtended = {args.eXtended}")


In [ ]:
%run ../scripts/argparse_example.py -X

---

**Add a single positional argument**

Let's add the other arguments one by one. First add a positional argument <filename>. The commandline will look something like this:
```
argparse_example.py [-X] my_file.txt
```
The positional argument is not optional.

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', '--eXtended', action='store_true', help="use the extended version")
parser.add_argument(dest='filename', action='store')
args = parser.parse_args()

print (f"args = {args}")
print (f"args.eXtended = {args.eXtended}")

print (f"args.filename = {args.filename}")


In [ ]:
%run ../scripts/argparse_example.py a_filename.txt

The ``dest`` argument to ``add_argument()`` specifies the name of the attribute where the result will be saved. The ``action='store'`` argument specifies the processing to be done and 'store' is the default. We can have more positional arguments and that is defined by the ``nargs=`` argument. In the following example there can be any number of filenames given.

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', '--eXtended', action='store_true', help="use the extended version")
parser.add_argument(dest='filenames', metavar="filename", nargs='*')  # nargs can be '?' or a number
args = parser.parse_args()

print (f"args = {args}")
print (f"args.eXtended = {args.eXtended}")

print (f"args.filenames = {args.filenames}")


In [ ]:
%run ../scripts/argparse_example.py -h

In [ ]:
%run ../scripts/argparse_example.py a_filename.txt

---

**Add an optional argument that takes a parameter**

Optional arguments can have a short and a long form as seen above with ``-X`` and ``--eXtended``. The options both need to be specified in the ``add_argument()`` method. When an option takes an argument, use ``action='store'`` instead of ``'store_true'``.

The difference between an option without and with an argument is in the ``action=`` argument of the ``add_argument()`` method. As shown below the argument can be separated from the option with a space or with the ``'='`` sign.

```
argparse_exaple.py [-X] [--log logfile.log] my_file.txt
```

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', '--eXtended', action='store_true', help="use the extended version")
parser.add_argument('--log', action='store', help="send logging messages to the specified file")
parser.add_argument(dest='filenames', metavar="filename", nargs='*')
args = parser.parse_args()

print (f"args = {args}")
print (f"args.eXtended = {args.eXtended}")
print (f"args.log = {args.log}")
print (f"args.filenames = {args.filenames}")


In [ ]:
%run ../scripts/argparse_example.py -h

In [ ]:
%run ../scripts/argparse_example.py --log logfile.log a_filename.txt

In [ ]:
%run ../scripts/argparse_example.py --log=logfile.log a_filename.txt

In [ ]:
%run ../scripts/argparse_example.py -X a_filename.txt

In the above example the ``--log`` option was not given and the ``args.log`` variable will be ``None`` in your Python script. To test for this option do for instance

```
if args.log:
    logging.basicConfig(filename=args.log, filemode='w')
```

---

**Special cases**

The last example shows how to have options with parameters of a specific type and parameters that can only have a limited set of values, i.e. choices. We add a logging level as an option with choices and a ``'max'`` option which takes an integer.

The commandline usage:

```
argparse_example.py [-X] [--log <logfile>] [--level <level>] [--max <number>] a_filename.txt
```

In [ ]:
%%script bash
cat > ../scripts/argparse_example.py
import argparse
import logging

parser = argparse.ArgumentParser(description="An example for parsing commandline arguments")

parser.add_argument('-X', '--eXtended', action='store_true', help="use the extended version")
parser.add_argument('--log', action='store', 
                    metavar='logfile',
                    help="send logging messages to the specified file")
parser.add_argument('--level', 
                    choices=('DEBUG', 'INFO', 'WARNING', 'ERROR'),
                    help='set the logging level')
parser.add_argument('--max', type=int, help='give the maximum number of ....')
parser.add_argument(dest='filenames', metavar="filename", nargs='*')

args = parser.parse_args()

print (f"args = {args}")
print (f"args.eXtended = {args.eXtended}")
print (f"args.log = {args.log}")
print (f"args.level = {args.level}")
print (f"args.max = {args.max} (type={type(args.max)})")
print (f"args.filenames = {args.filenames}")

log_level = args.level or 'INFO'

if args.log:
    logging.basicConfig(filename=args.log, filemode='w', level=log_level)
else:
    logging.basicConfig(level=log_level)

logging.debug('This is a debug logging message')
logging.info('This is a info logging message')
logging.warning('This is a warning logging message')
logging.error('This is a error logging message')
logging.info(f"The maximum number of ... is {args.max})")

In [ ]:
%run ../scripts/argparse_example.py -h

In [ ]:
reload(logging)

In [ ]:
%run ../scripts/argparse_example.py --log out.log --level=DEBUG a_filename.txt --max 23

### Summary

The ``argparse`` module provides an easy way to process command line arguments. We have seen the following examples:

* single optional arguments, e.g. ``-X`` or ``--eXtended``, use ``action='store_true'`` or ``action='store_false'``
* optional arguments with parameters, e.g. ``-o out.txt`` or ``--output-file out.txt``, use ``action='store'`` (default)
* optional arguments can also have a limited number of allowed values, i.e. ``choices``
* optional parameters and arguments can have a type
* positional arguments with ``dest='<attribute_name>'`` and/or ``nargs='[#|*|?]'``

The ``argparse`` module provides automatic help when using the ``-h`` option.

<div style="text-align: right"><button>[⇧ Go to top ⇧](#top_cell)</button></div>